In [1]:
from OptionStrategy import *

# Show the plots in the notebook.
%matplotlib inline
# Set the plots size.
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 12, 8
import matplotlib.pyplot as plt

import plotly.tools as tls
import plotly.plotly as py
import cufflinks as cf

Read the contracts.json file 

In [2]:
path_to_contracts_json = './testing_files/Contracts.json'
df_contracts = pd.read_json(path_to_contracts_json).dropna(axis=1, how='all').set_index('ConId')
#df_contracts.sort_values('LocalSymbol')

Let's buy a contract.

In [3]:
option = OptionOperation.from_ConId(contracts=df_contracts, ConID=198003954, position=Position.Long,
                                    premium=20)
print (option)

1 Long ES June-16 Put 2010 at 1000


Now, we can build an strategy.

In [4]:
strategy = OptionStrategy()
strategy.add(option)

option_2 = OptionOperation.from_ConId(contracts=df_contracts, ConID=198003965, position=Position.Short,
                                    premium=40)
strategy.add(option_2)

option_3 = OptionOperation.from_ConId(contracts=df_contracts, ConID=215521192, position=Position.Short,
                                    premium=40)
strategy.add(option_3)

option_4 = OptionOperation.from_ConId(contracts=df_contracts, ConID=198003244, position=Position.Long,
                                    premium=20)
strategy.add(option_4)

In [5]:
print(strategy)

1 Short ES June-16 Call 2055 at 2000
1 Long ES June-16 Put 2010 at 1000
1 Long ES June-16 Call 2070 at 1000
1 Short ES June-16 Put 2030 at 2000


Let's see the options 

In [6]:
col_names = []
for option in strategy.options.values():
    name = '{} {}{}'.format(option.position.name, option.strike_price, option.option_type.name)
    col_names.append(name)

In [7]:
strategy_profit_loss = {}
for price in range(2000, 2085, 5):
    profit_loss = []
    for option in strategy.options.values():
        profit_loss.append(option.profit_loss_at(price))
    #profit_loss.append(strategy.profit_loos_at(price))
    strategy_profit_loss[price] = profit_loss
    
df = pd.DataFrame(strategy_profit_loss).transpose()
df.columns = col_names

In [8]:
df.iplot(kind='scatter', width=2, colorscale="dflt", theme='ggplot', filename='testing_cufflinks/OptionStrategy')

And now the strategy itself

In [9]:
strategy_profit_loss = {}
for price in range(2000, 2085, 5):
    strategy_profit_loss[price] = strategy.profit_loss_at(price)
    
df_strategy = pd.Series(strategy_profit_loss)#.transpose()
df_strategy.name = 'Strategy'

In [10]:
df_strategy.iplot(kind='scatter', width=2, colorscale="dflt", theme='ggplot',
                  filename='testing_cufflinks/Strategy')